In [79]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score,precision_score
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('Data_08_02_2024.xlsx')

In [3]:
df.head()

,business_id,product_a,product_b,hard_closure,soft_closure,month_1,month_2,month_3,month_4,month_5,...,call_last_3_months,call_last_6_months,call_reason_account_setup_changes,call_reason_online_equipment,call_reason_other,call_reason_standalone_equipment,call_reason_transaction_batch_funding,Industry,Yrs_with_Chase,chargeback_txn_cnt
0,1000089098,1,0,0,1,0.00,4790.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Transportation, Communication, Utilities",NaN,1.0
1,1000089099,1,0,0,0,158.00,2263.00,5536.00,115.00,163.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business Services,35.3,NaN
2,1000089100,1,0,0,1,0.00,1165.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Construction,31.7,NaN
3,1000089101,1,0,0,0,1083.15,1052.97,729.71,426.27,992.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business Services,24.0,NaN
4,1000089102,1,0,1,0,0.00,0.00,0.00,3611.54,3390.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Construction,60.5,NaN


In [78]:
for i in zip(df,df.isna().mean()):
    print(i)

('product_a', 0.0)
('product_b', 0.0)
('hard_closure', 0.0)
('soft_closure', 0.0)
('month_1', 0.0)
('month_2', 0.0)
('month_3', 0.0)
('month_4', 0.0)
('month_5', 0.0)
('month_6', 0.0)
('month_7', 0.0)
('month_8', 0.0)
('month_9', 0.0)
('month_10', 0.0)
('month_11', 0.0)
('month_12', 0.0)
('month_13', 0.0)
('month_14', 0.0)
('month_15', 0.0)
('month_16', 0.0)
('month_17', 0.0)
('month_18', 0.0)
('month_19', 0.0)
('month_20', 0.0)
('month_21', 0.0)
('month_22', 0.0)
('month_23', 0.0)
('month_24', 0.0)
('month_25', 0.0)
('month_26', 0.0)
('month_27', 0.0)
('month_28', 0.0)
('month_29', 0.0)
('month_30', 0.0)
('month_31', 0.0)
('month_32', 0.0)
('month_33', 0.0)
('month_34', 0.0)
('month_35', 0.0)
('month_36', 0.0)
('month_37', 0.0)
('tpv_first_30_days', 0.0)
('tpv_first_60_days', 0.0)
('tpv_first_90_days', 0.0)
('tpv_first_120_days', 0.0)
('tpv_last_30_days', 0.0)
('tpv_last_60_days', 0.0)
('tpv_last_90_days', 0.0)
('tpv_last_120_days', 0.0)
('avg_monthly_tpv', 0.0)
('no_of_active_months'

In [5]:
df_copy = df.copy()

In [6]:
df.shape

(66077, 186)

In [7]:
threshold = len(df) * 0.20 # Calculate 20% of the total number of rows
dropped_columns = list(df.columns[df.isna().sum() > threshold])
df = df.dropna(thresh=threshold, axis=1)
print("Columns deleted:", dropped_columns)

Columns deleted: ['comp_month_1', 'comp_month_2', 'comp_month_3', 'comp_month_4', 'comp_month_5', 'comp_month_6', 'comp_month_7', 'comp_month_8', 'comp_month_9', 'comp_month_10', 'comp_month_11', 'comp_month_12', 'comp_month_13', 'comp_month_14', 'comp_month_15', 'comp_month_16', 'comp_month_17', 'comp_month_18', 'comp_month_19', 'comp_month_20', 'comp_month_21', 'comp_month_22', 'comp_month_23', 'comp_month_24', 'comp_month_25', 'comp_month_26', 'comp_month_27', 'comp_month_28', 'comp_month_29', 'comp_month_30', 'comp_month_31', 'comp_month_32', 'comp_month_33', 'comp_month_34', 'comp_month_35', 'comp_month_36', 'comp_month_37', 'comp_tpv_last_30_days', 'comp_tpv_last_60_days', 'comp_tpv_last_90_days', 'comp_tpv_last_120_days', 'comp_avg_monthly_tpv', 'comp_active_months', 'sales_volume', 'employees_total', 'Business_Age_Yrs', 'economic_area_of_influence', 'status_indicator', 'manufacturing_indicator', 'publicprivate_indicator', 'legal_status', 'hierarchy_code', 'number_of_family_memb

In [8]:
df.shape

(66077, 148)

In [9]:
# status_indicator,economic_area_of_influence,legal_status,hierarchy_code

In [10]:
df.drop(columns='business_id',inplace=True)

In [11]:
y = df[['hard_closure' ,'soft_closure']]
X = df.drop(columns=['hard_closure' ,'soft_closure'])

In [12]:
y

,hard_closure,soft_closure
0,0,1
1,0,0
2,0,1
3,0,0
4,1,0
...,...,...
66072,0,0
66073,0,1
66074,0,1
66075,0,0


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import pandas as pd

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

preprocessor.fit(X)
processed_data = preprocessor.transform(X)

X = pd.DataFrame(processed_data)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [53]:
y['hard_closure'].value_counts()

0    59187
1     6890
Name: hard_closure, dtype: int64

In [55]:
y['hard_closure'].value_counts(normalize=True)*100

0    89.572771
1    10.427229
Name: hard_closure, dtype: float64

In [52]:
y['soft_closure'].value_counts(normalize=True)*100

1    61.929264
0    38.070736
Name: soft_closure, dtype: float64

In [57]:
y_train['soft_closure']

18716    1
56011    0
6400     0
27951    1
43851    0
        ..
37194    1
6265     1
54886    1
860      1
15795    1
Name: soft_closure, Length: 52861, dtype: int64

In [80]:
from sklearn.metrics import classification_report,accuracy_score
from sklearn import ensemble,neighbors,tree

from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# https://scikit-learn.org/stable/modules/multiclass.html
MLA = [
    LogisticRegression(),
    ensemble.RandomForestClassifier(random_state = 1,n_jobs=-1),
    ensemble.ExtraTreesClassifier(n_jobs=-1),
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
]

MLA_columns = ['MLA_names', 'Hard_Test_Accuracy','Hard_Test_F1','Hard_Test_Precision',
                            'Soft_Test_Accuracy','Soft_Test_F1','Soft_Test_Precision']

classifiers = {}
def apply_models(X_train,y_train,X_test,y_test):
    MLA_compare = pd.DataFrame(columns = MLA_columns)

    row_index = 0
    for alg in tqdm(MLA):
        try:
            MLA_name = alg.__class__.__name__
            MLA_compare.loc[row_index, 'MLA_names'] = MLA_name

            multi_target_forest = MultiOutputClassifier(alg, n_jobs=-1)
            multi_target_forest.fit(X_train, y_train)
            
            y_pred = multi_target_forest.predict(X_test)
           
            classifiers[MLA_name] = multi_target_forest
            ############## Hard closure results ###########
            acc = accuracy_score(y_test['hard_closure'],y_pred[:,0])
            f1 = f1_score(y_test['hard_closure'],y_pred[:,0])
            precision = precision_score(y_test['hard_closure'],y_pred[:,0])
            
            MLA_compare.loc[row_index, 'Hard_Test_Accuracy'] = acc
            MLA_compare.loc[row_index, 'Hard_Test_F1'] = f1
            MLA_compare.loc[row_index, 'Hard_Test_Precision'] = precision
            
            ########## Soft closure results ############
            acc = accuracy_score(y_test['soft_closure'],y_pred[:,1])
            f1 = f1_score(y_test['soft_closure'],y_pred[:,1])
            precision = precision_score(y_test['soft_closure'],y_pred[:,1])
            
            MLA_compare.loc[row_index, 'Soft_Test_Accuracy'] = acc
            MLA_compare.loc[row_index, 'Soft_Test_F1'] = f1
            MLA_compare.loc[row_index, 'Soft_Test_Precision'] = precision
            
            row_index += 1
        except:
            print(type(alg).__name__)
    return MLA_compare

In [81]:
apply_models(X_train,y_train,X_test,y_test)

  0%|          | 0/5 [00:00<?, ?it/s]

LogisticRegression


,MLA_names,Hard_Test_Accuracy,Hard_Test_F1,Hard_Test_Precision,Soft_Test_Accuracy,Soft_Test_F1,Soft_Test_Precision
0,RandomForestClassifier,0.906023,0.268551,0.640449,0.999849,0.999878,0.999755
1,ExtraTreesClassifier,0.900045,0.202776,0.533333,0.970642,0.976376,0.971643
2,DecisionTreeClassifier,0.856916,0.331566,0.3154,1.0,1.0,1.0
3,ExtraTreeClassifier,0.847004,0.266328,0.259547,0.89437,0.914722,0.913272


# Prediction

In [33]:
classifiers.keys()

dict_keys(['RandomForestClassifier', 'ExtraTreesClassifier', 'DecisionTreeClassifier', 'ExtraTreeClassifier'])

In [58]:
classifiers['DecisionTreeClassifier']

MultiOutputClassifier(estimator=DecisionTreeClassifier(), n_jobs=-1)

In [61]:
test_data = df.sample(10)

In [62]:
test_data

,product_a,product_b,hard_closure,soft_closure,month_1,month_2,month_3,month_4,month_5,month_6,...,90plus_days_delinquent_bc_past_6_months,90plus_days_delinquent_bs_past_6_months,30plus_days_delinquent_bc_past_12_months,30plus_days_delinquent_bs_past_12_months,60plus_days_delinquent_bc_past_12_months,60plus_days_delinquent_bs_past_12_months,90plus_days_delinquent_bc_past_12_months,90plus_days_delinquent_bs_past_12_months,Industry,Yrs_with_Chase
25903,1,0,0,1,0.0,2.00,1099.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Retail,NaN
8760,1,0,1,1,0.0,454.50,362.0,479.0,499.0,566.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Farming Agriculture,2.6
29616,1,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare,NaN
29447,1,0,0,1,0.0,5.00,0.0,3100.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Personal Services,2.5
41785,1,0,0,1,0.0,0.00,1220.0,185.0,150.0,540.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Healthcare,1.9
52660,1,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Construction,1.4
8203,1,0,0,1,0.0,0.00,0.0,1737.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Transportation, Communication, Utilities",NaN
30047,1,0,1,1,0.0,0.00,360.5,1739.0,14430.0,12058.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B2B,NaN
27093,1,0,0,1,0.0,7631.31,2537.9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Business Services,NaN
9216,1,0,0,0,0.0,0.00,0.0,35.2,324.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eating Drinking Places,1.8


In [63]:
y_test_data = test_data[['hard_closure' ,'soft_closure']]
X_test_data = test_data.drop(columns=['hard_closure' ,'soft_closure'])

In [64]:
X_test_data

,product_a,product_b,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,...,90plus_days_delinquent_bc_past_6_months,90plus_days_delinquent_bs_past_6_months,30plus_days_delinquent_bc_past_12_months,30plus_days_delinquent_bs_past_12_months,60plus_days_delinquent_bc_past_12_months,60plus_days_delinquent_bs_past_12_months,90plus_days_delinquent_bc_past_12_months,90plus_days_delinquent_bs_past_12_months,Industry,Yrs_with_Chase
25903,1,0,0.0,2.00,1099.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Retail,NaN
8760,1,0,0.0,454.50,362.0,479.0,499.0,566.0,271.8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Farming Agriculture,2.6
29616,1,0,0.0,0.00,0.0,0.0,0.0,0.0,250.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare,NaN
29447,1,0,0.0,5.00,0.0,3100.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Personal Services,2.5
41785,1,0,0.0,0.00,1220.0,185.0,150.0,540.0,410.0,521.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Healthcare,1.9
52660,1,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Construction,1.4
8203,1,0,0.0,0.00,0.0,1737.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Transportation, Communication, Utilities",NaN
30047,1,0,0.0,0.00,360.5,1739.0,14430.0,12058.0,16948.0,18118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B2B,NaN
27093,1,0,0.0,7631.31,2537.9,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Business Services,NaN
9216,1,0,0.0,0.00,0.0,35.2,324.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eating Drinking Places,1.8


In [65]:
X_test_data_processed = preprocessor.transform(X_test_data)
X_test_data_processed = pd.DataFrame(X_test_data_processed)

In [66]:
X_test_data_processed

,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,221
0,1.0,0.0,0.0,0.000001,0.000318,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.000253,0.000105,0.000156,0.000206,0.000201,0.000108,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000100,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.000003,0.000000,0.001010,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.000000,0.000353,0.000060,0.000062,0.000192,0.000163,0.000179,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.000000,0.000000,0.000566,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,1.0,0.0,0.0,0.000000,0.000104,0.000566,0.005953,0.004286,0.006750,0.006232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.004247,0.000734,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.000000,0.000000,0.000011,0.000134,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
clf = classifiers['DecisionTreeClassifier']
pred = clf.predict(X_test_data_processed)

In [68]:
pred

array([[0, 1],
       [1, 1],
       [0, 0],
       [0, 1],
       [0, 1],
       [0, 0],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 0]], dtype=int64)

In [69]:
pd.DataFrame(pred,columns=['hard_closure' ,'soft_closure'])

,hard_closure,soft_closure
0,0,1
1,1,1
2,0,0
3,0,1
4,0,1
5,0,0
6,0,1
7,1,1
8,0,1
9,0,0


run logistics regression with using hard and soft closure with the other variables to see which variables are significant. I should also try to find correlated variables. For instance you can use some of the variables to predict soft closure and hard closure. Also, after I run the logistic regression to find the coefficients and the significant variables, I should proceed to build the churn prediction model.